# Library Import
This section imports all the necessary libraries required for data processing, feature extraction, and cloud interaction.

In [39]:
from keras.models import load_model
import pandas as pd
import numpy as np
import json

# Loading the model

In [173]:
model = load_model('./model/best_model_nasa.keras')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 128)                 │           2,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,917 (27.02 KB)

 Trainable params: 2,305 (9.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,612 (18.02 KB)

# Creating the output catalog for the Lunar Test data

In [148]:
df = pd.read_csv('./data/test_lunar.csv')
df.shape

(36071397, 27)

In [149]:
df.head()

,Unnamed: 0,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),velocity(m/s),filename,network,station,location,channel,sampling_rate,...,total_energy,rms_value,peak_count,valley_count,fft_values,fft_freqs,autocorrelation,acceleration,jerk,cumulative_energy
0,0,1972-02-28 00:00:00.348000,0.000000,-1.088482e-14,xa.s12.00.mhz.1972-02-28HR00_evid00192,XA,S12,0,MHZ,6.625,...,6.139464e-14,3.274979e-10,71632,71632,(-4.716102943072456e-07-0j),0.000000,6.139464e-14,-1.807863e-14,3.290433e-14,0.000000e+00
1,1,1972-02-28 00:00:00.498943,0.150943,-1.361367e-14,xa.s12.00.mhz.1972-02-28HR00_evid00192,XA,S12,0,MHZ,6.625,...,6.139464e-14,3.274979e-10,71632,71632,(3.897736953158206e-07+9.6057431395568e-08j),0.000012,4.472243e-14,-1.311194e-14,5.207360e-14,2.292914e-29
2,2,1972-02-28 00:00:00.649887,0.301887,-1.484314e-14,xa.s12.00.mhz.1972-02-28HR00_evid00192,XA,S12,0,MHZ,6.625,...,6.139464e-14,3.274979e-10,71632,71632,(-2.706673032362094e-07+2.8195511816846415e-07j),0.000023,9.239345e-15,-2.358301e-15,6.832794e-14,5.354428e-29
3,3,1972-02-28 00:00:00.800830,0.452830,-1.432561e-14,xa.s12.00.mhz.1972-02-28HR00_evid00192,XA,S12,0,MHZ,6.625,...,6.139464e-14,3.274979e-10,71632,71632,(3.920114055364586e-07-2.065272850274259e-07j),0.000035,-2.261726e-14,7.515361e-15,5.061417e-14,8.566063e-29
4,4,1972-02-28 00:00:00.951774,0.603774,-1.257435e-14,xa.s12.00.mhz.1972-02-28HR00_evid00192,XA,S12,0,MHZ,6.625,...,6.139464e-14,3.274979e-10,71632,71632,(-9.573717207967279e-09+2.8507295917219967e-07j),0.000046,-3.260775e-14,1.292145e-14,1.992987e-14,1.130823e-28


In [193]:
temp = copy.deepcopy(df)
temp.drop(temp.columns[0], axis=1,inplace=True)
temp['fft_values'] = temp['fft_values'].apply(lambda x: np.complex128(x))
temp.drop(['time_abs(%Y-%m-%dT%H:%M:%S.%f)','network','station','location','npts','channel','sampling_rate','delta','calib'],axis=1,inplace=True)
temp.drop(['filename'],axis=1,inplace=True)
temp.head()

,time_rel(sec),velocity(m/s),mean_velocity,std_velocity,max_velocity,min_velocity,total_energy,rms_value,peak_count,valley_count,fft_values,fft_freqs,autocorrelation,acceleration,jerk,cumulative_energy
0,0.000000,-1.088482e-14,-8.238914e-13,3.274969e-10,1.191479e-08,-1.182754e-08,6.139464e-14,3.274979e-10,71632,71632,-4.716103e-07-0.000000e+ 00j,0.000000,6.139464e-14,-1.807863e-14,3.290433e-14,0.000000e+00
1,0.150943,-1.361367e-14,-8.238914e-13,3.274969e-10,1.191479e-08,-1.182754e-08,6.139464e-14,3.274979e-10,71632,71632,3.897737e-07+9.605743e- 08j,0.000012,4.472243e-14,-1.311194e-14,5.207360e-14,2.292914e-29
2,0.301887,-1.484314e-14,-8.238914e-13,3.274969e-10,1.191479e-08,-1.182754e-08,6.139464e-14,3.274979e-10,71632,71632,-2.706673e-07+2.819551e- 07j,0.000023,9.239345e-15,-2.358301e-15,6.832794e-14,5.354428e-29
3,0.452830,-1.432561e-14,-8.238914e-13,3.274969e-10,1.191479e-08,-1.182754e-08,6.139464e-14,3.274979e-10,71632,71632,3.920114e-07-2.065273e- 07j,0.000035,-2.261726e-14,7.515361e-15,5.061417e-14,8.566063e-29
4,0.603774,-1.257435e-14,-8.238914e-13,3.274969e-10,1.191479e-08,-1.182754e-08,6.139464e-14,3.274979e-10,71632,71632,-9.573717e-09+2.850730e- 07j,0.000046,-3.260775e-14,1.292145e-14,1.992987e-14,1.130823e-28


In [194]:
y_pred=model.predict(temp,verbose=2)
df['y_pred'] = (y_pred > 0.5).astype(int)  # Convert to 0 or 1

C:\Users\vitor\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:133: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


1127232/1127232 - 1064s - 944us/step


In [202]:
df.head()

,time_rel(sec),velocity(m/s),filename,mean_velocity,std_velocity,max_velocity,min_velocity,total_energy,rms_value,peak_count,valley_count,fft_values,fft_freqs,autocorrelation,acceleration,jerk,cumulative_energy,y_pred
0,0.000000,-1.088482e-14,xa.s12.00.mhz.1972-02-28HR00_evid00192,-8.238914e-13,3.274969e-10,1.191479e-08,-1.182754e-08,6.139464e-14,3.274979e-10,71632,71632,-4.716103e-07-0.000000e+ 00j,0.000000,6.139464e-14,-1.807863e-14,3.290433e-14,0.000000e+00,0
1,0.150943,-1.361367e-14,xa.s12.00.mhz.1972-02-28HR00_evid00192,-8.238914e-13,3.274969e-10,1.191479e-08,-1.182754e-08,6.139464e-14,3.274979e-10,71632,71632,3.897737e-07+9.605743e- 08j,0.000012,4.472243e-14,-1.311194e-14,5.207360e-14,2.292914e-29,0
2,0.301887,-1.484314e-14,xa.s12.00.mhz.1972-02-28HR00_evid00192,-8.238914e-13,3.274969e-10,1.191479e-08,-1.182754e-08,6.139464e-14,3.274979e-10,71632,71632,-2.706673e-07+2.819551e- 07j,0.000023,9.239345e-15,-2.358301e-15,6.832794e-14,5.354428e-29,0
3,0.452830,-1.432561e-14,xa.s12.00.mhz.1972-02-28HR00_evid00192,-8.238914e-13,3.274969e-10,1.191479e-08,-1.182754e-08,6.139464e-14,3.274979e-10,71632,71632,3.920114e-07-2.065273e- 07j,0.000035,-2.261726e-14,7.515361e-15,5.061417e-14,8.566063e-29,0
4,0.603774,-1.257435e-14,xa.s12.00.mhz.1972-02-28HR00_evid00192,-8.238914e-13,3.274969e-10,1.191479e-08,-1.182754e-08,6.139464e-14,3.274979e-10,71632,71632,-9.573717e-09+2.850730e- 07j,0.000046,-3.260775e-14,1.292145e-14,1.992987e-14,1.130823e-28,0


## Creating aditional features for the output catalog:
- identified_arrival_time_rel(sec): time_rel(s) in the first row considered an earthquake
- detection_duration(sec): time_rel(s) in the last row of the entire subset of a record
- selection_duration(sec): detection_duration(sec) - identified_arrival_time_rel(sec)
- features_at_detection: the features in the first row considered an earthquake
- file_original_size(kb): pandas.memory_usage() method on the complete subset of a detection
- file_selection_size(kb): pandas.memory_usage() method on the subset trimmed from the detection index to the end
- original_broadcast: file_original_size(kb) / transmission rate
- selection_broadcast: file_selection_size(kb) / transmission rate

- Data transmission rates:

    - Apollo 12: 51.2 kbps
    - Apollo 15: 85.6 kbps
    - Apollo 16: 85.6 kbps
    - InSight: 256 kbps

In [203]:
result = df[df['y_pred'] == 1].groupby('filename', as_index=False).nth(0).reset_index()
memory_usage_per_group = df[df['y_pred'] == 1].groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
result['file_selection_size(kb)'] = result['filename'].map(memory_usage_per_group)

features = result.drop(['file_selection_size(kb)'],axis=1)

json_list = []
for index, row in features.iterrows():
    row_dict = row.to_dict()
    
    # Convert complex numbers to strings
    for key, value in row_dict.items():
        if isinstance(value, complex):
            row_dict[key] = str(value)
    
    json_list.append(json.dumps(row_dict))
    
result = result[['filename', 'time_rel(sec)','velocity(m/s)','index','file_selection_size(kb)']].rename(columns={"index":"index_predict","time_rel(sec)":"identified_arrival_time_rel(sec)"})

result2 = df.groupby('filename', as_index=False).tail(1).reset_index()[['filename','index', 'time_rel(sec)']].rename(
    columns={"index": "index_tail", "time_rel(sec)": "detection_duration(sec)"}
)
memory_usage_per_group = df.groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
result2['file_original_size(kb)'] = result2['filename'].map(memory_usage_per_group)

C:\Users\vitor\AppData\Local\Temp\ipykernel_35148\3447552071.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  memory_usage_per_group = df[df['y_pred'] == 1].groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
C:\Users\vitor\AppData\Local\Temp\ipykernel_35148\3447552071.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  memory_usage_per_group = df.groupby('filename').apply(lambd

In [204]:
result3 = pd.concat([result, result2.drop(['filename'],axis=1)],axis=1)
result3['selection_duration'] = result3['detection_duration(sec)'] - result3['identified_arrival_time_rel(sec)']
result3['features']=json_list
conditions = [
    result3['filename'].str.contains('s12'),
    result3['filename'].str.contains('s15'),
    result3['filename'].str.contains('s16')
]
values = [51.2, 85.6, 85.6]
result3['transmission_speed'] = np.select(conditions, values, default=256)
result3['original_broadcast'] = result3['file_original_size(kb)'] / result3['transmission_speed']
result3['selection_broadcast'] = result3['file_selection_size(kb)'] / result3['transmission_speed']
result3.drop(['index_predict','index_tail','transmission_speed'],inplace=True,axis=1)
result3

,filename,identified_arrival_time_rel(sec),velocity(m/s),file_selection_size(kb),detection_duration(sec),file_original_size(kb),selection_duration,features,original_broadcast,selection_broadcast
0,xa.s12.00.mhz.1972-02-28HR00_evid00192,42540.226415,3.746221e-11,70613127,86402.566038,139097574,43862.339623,"{""index"": 281829, ""time_rel(sec)"": 42540.22641...",2.716749e+06,1.379163e+06
1,xa.s12.00.mhz.1970-05-24HR00_evid00028,41189.735849,1.537547e-16,72785547,86401.509434,139095873,45211.773585,"{""index"": 845300, ""time_rel(sec)"": 41189.73584...",2.716716e+06,1.421593e+06
2,xa.s12.00.mhz.1975-05-20HR00_evid00652,43201.358491,-1.695787e-10,69548787,86402.566038,139097574,43201.207547,"{""index"": 1431038, ""time_rel(sec)"": 43201.3584...",2.716749e+06,1.358375e+06
3,xa.s12.00.mhz.1972-11-19HR00_evid00335,43582.188679,-1.051196e-10,68934969,86402.113208,139096845,42819.924528,"{""index"": 2005979, ""time_rel(sec)"": 43582.1886...",2.716735e+06,1.346386e+06
4,xa.s12.00.mhz.1970-07-27HR00_evid00039,42366.037736,-1.025104e-11,70893549,86402.566038,139097574,44036.528302,"{""index"": 2570337, ""time_rel(sec)"": 42366.0377...",2.716749e+06,1.384640e+06
...,...,...,...,...,...,...,...,...,...,...
59,xa.s12.00.mhz.1970-09-09HR00_evid00043,44589.886792,-3.834688e-10,67315617,86403.924528,139099761,41814.037736,"{""index"": 33504769, ""time_rel(sec)"": 44589.886...",2.716792e+06,1.314758e+06
60,xa.s12.00.mhz.1972-05-19HR00_evid00228,43994.264151,6.326557e-12,68268420,86400.150943,139093686,42405.886792,"{""index"": 34073250, ""time_rel(sec)"": 43994.264...",2.716674e+06,1.333368e+06
61,xa.s12.00.mhz.1972-12-06HR00_evid00342,44790.339623,-5.949950e-13,66986109,86399.698113,139092957,41609.358491,"{""index"": 34650926, ""time_rel(sec)"": 44790.339...",2.716659e+06,1.308322e+06
62,xa.s12.00.mhz.1971-06-11HR00_evid00096,44527.396226,1.427021e-12,67414032,86402.566038,139097574,41875.169811,"{""index"": 35221583, ""time_rel(sec)"": 44527.396...",2.716749e+06,1.316680e+06


In [205]:
result3.to_csv("./lunar_test_output_catalog.csv",index=False)

# Creating the output catalog for the Mars Test data

In [180]:
df_mars = pd.read_csv('./data/test_mars.csv')
df_mars.drop(df_mars.columns[0], axis=1,inplace=True)
df_mars['fft_values'] = df_mars['fft_values'].apply(lambda x: np.complex128(x))
df_mars.drop(['time(%Y-%m-%dT%H:%M:%S.%f)','network','station','location','npts','channel','sampling_rate','delta','calib'],axis=1,inplace=True)
df_mars = df_mars.rename(columns={'velocity(c/s)': 'velocity(m/s)','rel_time(sec)':'time_rel(sec)'}) 
df_mars.head()
temp = copy.deepcopy(df_mars)
temp.drop(['filename'],axis=1,inplace=True)

In [183]:
y_pred=model.predict(temp,verbose=2)
df_mars['y_pred'] = (y_pred > 0.5).astype(int)

C:\Users\vitor\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:133: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


20250/20250 - 18s - 891us/step


In [188]:
df_mars.head()

,time_rel(sec),velocity(m/s),filename,mean_velocity,std_velocity,max_velocity,min_velocity,total_energy,rms_value,peak_count,valley_count,fft_values,fft_freqs,autocorrelation,acceleration,jerk,cumulative_energy,y_pred
0,0.00,0.000000,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033,-0.106292,163.525782,2894.658691,-2175.483398,1.925330e+09,163.525816,21827,21826,-7.653055e+03-0.000000e+ 00j,0.000000,1.925330e+09,0.001337,-0.015385,0.000000e+00,0
1,0.05,0.000067,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033,-0.106292,163.525782,2894.658691,-2175.483398,1.925330e+09,163.525816,21827,21826,5.032086e+05+4.519214e+ 04j,0.000278,1.877502e+09,0.000568,-0.179246,1.117610e-10,0
2,0.10,0.000057,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033,-0.106292,163.525782,2894.658691,-2175.483398,1.925330e+09,163.525816,21827,21826,-1.293472e+06-3.854668e+ 05j,0.000556,1.860496e+09,-0.016587,0.155938,3.041675e-10,0
3,0.15,-0.001592,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033,-0.106292,163.525782,2894.658691,-2175.483398,1.925330e+09,163.525816,21827,21826,-7.829146e+05+4.120607e+ 05j,0.000833,1.875679e+09,0.016162,0.657648,6.373614e-08,0
4,0.20,0.001673,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033,-0.106292,163.525782,2894.658691,-2175.483398,1.925330e+09,163.525816,21827,21826,1.182444e+06+8.483111e+ 05j,0.001111,1.848085e+09,0.049177,-0.520651,1.970583e-07,0


## Creating aditional features for the output catalog:
- identified_arrival_time_rel(sec): time_rel(s) in the first row considered an earthquake
- detection_duration(sec): time_rel(s) in the last row of the entire subset of a record
- selection_duration(sec): detection_duration(sec) - identified_arrival_time_rel(sec)
- features_at_detection: the features in the first row considered an earthquake
- file_original_size(kb): pandas.memory_usage() method on the complete subset of a detection
- file_selection_size(kb): pandas.memory_usage() method on the subset trimmed from the detection index to the end
- original_broadcast: file_original_size(kb) / transmission rate
- selection_broadcast: file_selection_size(kb) / transmission rate

- Data transmission rates:

    - Apollo 12: 51.2 kbps
    - Apollo 15: 85.6 kbps
    - Apollo 16: 85.6 kbps
    - InSight: 256 kbps

In [189]:
result = df_mars[df_mars['y_pred'] == 1].groupby('filename', as_index=False).nth(0).reset_index()
memory_usage_per_group = df_mars[df_mars['y_pred'] == 1].groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
result['file_selection_size(kb)'] = result['filename'].map(memory_usage_per_group)

features = result.drop(['file_selection_size(kb)'],axis=1)

json_list = []
for index, row in features.iterrows():
    row_dict = row.to_dict()
    
    # Convert complex numbers to strings
    for key, value in row_dict.items():
        if isinstance(value, complex):
            row_dict[key] = str(value)
    
    json_list.append(json.dumps(row_dict))
    
result = result[['filename', 'time_rel(sec)','velocity(m/s)','index','file_selection_size(kb)']].rename(columns={"index":"index_predict","time_rel(sec)":"identified_arrival_time_rel(sec)"})

result2 = df_mars.groupby('filename', as_index=False).tail(1).reset_index()[['filename','index', 'time_rel(sec)']].rename(
    columns={"index": "index_tail", "time_rel(sec)": "detection_duration(sec)"}
)
memory_usage_per_group = df_mars.groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
result2['file_original_size(kb)'] = result2['filename'].map(memory_usage_per_group)

C:\Users\vitor\AppData\Local\Temp\ipykernel_35148\638689904.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  memory_usage_per_group = df_mars[df_mars['y_pred'] == 1].groupby('filename').apply(lambda group: group.memory_usage(deep=True).sum())
C:\Users\vitor\AppData\Local\Temp\ipykernel_35148\638689904.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  memory_usage_per_group = df_mars.groupby('filename'

In [190]:
result3 = pd.concat([result, result2.drop(['filename'],axis=1)],axis=1)
result3['selection_duration'] = result3['detection_duration(sec)'] - result3['identified_arrival_time_rel(sec)']
result3['features']=json_list
conditions = [
    result3['filename'].str.contains('s12'),
    result3['filename'].str.contains('s15'),
    result3['filename'].str.contains('s16')
]
values = [51.2, 85.6, 85.6]
result3['transmission_speed'] = np.select(conditions, values, default=256)
result3['original_broadcast'] = result3['file_original_size(kb)'] / result3['transmission_speed']
result3['selection_broadcast'] = result3['file_selection_size(kb)'] / result3['transmission_speed']
result3.drop(['index_predict','index_tail','transmission_speed'],inplace=True,axis=1)
result3

,filename,identified_arrival_time_rel(sec),velocity(m/s),file_selection_size(kb),detection_duration(sec),file_original_size(kb),selection_duration,features,original_broadcast,selection_broadcast
0,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033,712.30,10.211641,13900924,3599.95,17568000,2887.65,"{""index"": 14246, ""time_rel(sec)"": 712.3, ""velo...",68625.000000,54300.484375
1,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041,381.90,-33.574017,15141908,3599.95,17568000,3218.05,"{""index"": 79638, ""time_rel(sec)"": 381.9, ""velo...",68625.000000,59148.078125
2,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001,0.10,0.007310,9047520,3599.90,17567756,3599.80,"{""index"": 144002, ""time_rel(sec)"": 0.1, ""veloc...",68624.046875,35341.875000
3,XB.ELYSE.02.BHV.2022-04-09HR22_evid0002,324.45,263.195374,15605508,3599.95,17568000,3275.50,"{""index"": 222488, ""time_rel(sec)"": 324.45, ""ve...",68625.000000,60959.015625
4,XB.ELYSE.02.BHV.2019-09-21HR03_evid0032,116.55,-2496.125732,16999236,3599.95,17568000,3483.40,"{""index"": 290330, ""time_rel(sec)"": 116.55, ""ve...",68625.000000,66403.265625
5,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007,0.15,-0.023623,15868052,3599.95,17568000,3599.80,"{""index"": 360002, ""time_rel(sec)"": 0.15, ""velo...",68625.000000,61984.578125
6,XB.ELYSE.02.BHV.2021-10-11HR23_evid0011,310.45,-352.255646,15870980,3599.95,17568000,3289.50,"{""index"": 438208, ""time_rel(sec)"": 310.45, ""ve...",68625.000000,61996.015625
7,XB.ELYSE.02.BHV.2021-05-02HR01_evid0017,691.35,-188.084656,14033416,3599.95,17568000,2908.60,"{""index"": 517826, ""time_rel(sec)"": 691.35, ""ve...",68625.000000,54818.031250
8,XB.ELYSE.02.BHV.2019-07-26HR12_evid0034,712.30,10.211641,13900924,3599.95,17568000,2887.65,"{""index"": 590245, ""time_rel(sec)"": 712.3, ""vel...",68625.000000,54300.484375


In [191]:
result3.to_csv("./mars_test_output_catalog.csv",index=False)